In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

###SILVER LAYER SCRIPT

### Read calendar data

In [0]:
df_cal= spark.read.format("csv").option("header",True).option("inferSchema",True).load('abfss://bronze@mstoragedatalake.dfs.core.windows.net/AdventureWorks_Calendar')

In [0]:
df_cus= spark.read.format("csv").option("header",True).option("inferSchema",True).load('abfss://bronze@mstoragedatalake.dfs.core.windows.net/AdventureWorks_Customers')

In [0]:
df_procat= spark.read.format("csv").option("header",True).option("inferSchema",True).load('abfss://bronze@mstoragedatalake.dfs.core.windows.net/AdventureWorks_Product_Categories')

In [0]:
df_pro= spark.read.format("csv").option("header",True).option("inferSchema",True).load('abfss://bronze@mstoragedatalake.dfs.core.windows.net/AdventureWorks_Products')

In [0]:
df_ret= spark.read.format("csv").option("header",True).option("inferSchema",True).load('abfss://bronze@mstoragedatalake.dfs.core.windows.net/AdventureWorks_Returns')

In [0]:
df_sal_2015= spark.read.format("csv").option("header",True).option("inferSchema",True).load('abfss://bronze@mstoragedatalake.dfs.core.windows.net/AdventureWorks_Sales_2015')

In [0]:
df_sal_2016= spark.read.format("csv").option("header",True).option("inferSchema",True).load('abfss://bronze@mstoragedatalake.dfs.core.windows.net/AdventureWorks_Sales_2016')

In [0]:
df_sal_2017= spark.read.format("csv").option("header",True).option("inferSchema",True).load('abfss://bronze@mstoragedatalake.dfs.core.windows.net/AdventureWorks_Sales_2017')

In [0]:
df_ter= spark.read.format("csv").option("header",True).option("inferSchema",True).load('abfss://bronze@mstoragedatalake.dfs.core.windows.net/AdventureWorks_Territories')

In [0]:
df_pro_subcat= spark.read.format("csv").option("header",True).option("inferSchema",True).load('abfss://bronze@mstoragedatalake.dfs.core.windows.net/Product_Subcategories')

In [0]:
### TRANSFORMATIONS

In [0]:
df_cal= df_cal.withColumn('Month',month(col('Date'))).withColumn('Year',year(col('Date')))
df_cal_mon.display()

In [0]:
df_cal.write.format('parquet').mode('append').option("path","abfss://silver@mstoragedatalake.dfs.core.windows.net/AdventureWorks_Calendar").save()

In [0]:
df_cus= df_cus.withColumn('fullName',concat(col('Prefix'),lit(' '),col('FirstName'),lit(' '),col('LastName'))).display()

In [0]:
df_cus= df_cus.withColumn('fullName',concat_ws(' ',col('Prefix'),col('FirstName'),col('LastName')))

In [0]:
df_cus.write.format('parquet').mode('append').option("path","abfss://silver@mstoragedatalake.dfs.core.windows.net/AdventureWorks_Customer").save()

In [0]:
df_pro= df_pro.withColumn('ProductSKU',split(col('ProductSKU'),'-')[0])\
    .withColumn('ProductName',split(col('ProductName'),' ')[0])

In [0]:
df_pro.write.format('parquet').mode('append').option("path","abfss://silver@mstoragedatalake.dfs.core.windows.net/AdventureWorks_Products").save()

In [0]:
df_sal_2015= df_sal_2015.withColumn('StockDate',to_timestamp('StockDate'))

In [0]:
df_sal_2015= df_sal_2015.withColumn('OrderNumber',regexp_replace(col('OrderNumber'),'S','T'))

In [0]:
df_sal_2015= df_sal_2015.withColumn('multiply',col('OrderLineItem')*col('OrderQuantity'))

In [ ]:
df_sal_2015.groupBy('OrderDate').agg(count('OrderNumber').alias('total_order'))

In [0]:
df_sal_2015.write.format('parquet').mode('append').option("path","abfss://silver@mstoragedatalake.dfs.core.windows.net/AdventureWorks_Sales").save()